## 从头开始构建vgg
本文目的是根据Keras来构建下vgg16

In [1]:
import os, sys
current_directory = os.getcwd()
LESSON_HOME_DIR = current_directory
DATA_HOME_DIR = os.path.join(current_directory,"../data/redux")
print(DATA_HOME_DIR)
print(LESSON_HOME_DIR)

/home/carnd/fast-ai/lesson1/../data/redux
/home/carnd/fast-ai/lesson1


In [ ]:
%cd $DATA_HOME_DIR
!tree -d

In [ ]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda,Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

In [ ]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

In [ ]:
print(vgg_mean.shape,vgg_mean)

下面开始是Vgg16的模型
![](vgg16_original.png)

通过5个卷积层提取出特征，然后再通过一个全连接做分类，下面我们开始构建的

In [ ]:
import keras
from keras import backend as K
print(keras.__version__)

In [ ]:
print(K.image_data_format())
print(K.image_dim_ordering()) # image_dim_ordering 和 image_data_format 是一致的

In [ ]:
# K.set_image_data_format("channels_last")
# K.set_image_dim_ordering("tf")

In [ ]:
def ConvBlock(model, layers, filters):
    """
        Adds a specified number of ZeroPadding and Covolution layers
        to the model, and a MaxPooling layer at the very end.

        Args:
            layers (int):   The number of zero padded convolution layers
                            to be added to the model.
            filters (int):  The number of convolution filters to be 
                            created for each layer.
    """
    for i in range(layers):
        # the dimensionality of the output space
        model.add(Conv2D(filters,kernel_size=(3,3))) # padding='valid'
        model.add(Activation('relu'))          
        model.add(MaxPooling2D(pool_size=(2, 2)))
    return model

In [ ]:
model = Sequential()
# 第一层是一个对输入预处理的层，将图片0-255减去一个平均数，然后将 rgb->bgr
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
ConvBlock(model,2, 64)

In [ ]:
model.layers

In [ ]:
for layer in model.layers:
    print(layer.input)

## keras自带的VGG16模型

In [2]:
from keras import applications
from keras import backend as K
# 我们看下自带的模型是如何的

Using TensorFlow backend.


In [3]:
# K.set_image_data_format("channels_last")

In [4]:
print(K.image_data_format())
print(K.image_dim_ordering())

channels_last
tf


In [16]:
image_witdth = 150
image_height = 150
input_shape = (image_witdth, image_height, 3)

In [17]:
vgg16 = applications.VGG16(include_top=False,weights='imagenet',input_shape=input_shape)

In [18]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/sample' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
utils_path = current_directory + "/../utils"

/home/carnd/fast-ai/data/redux


In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
gen = ImageDataGenerator(featurewise_center=True) #featurewise_center: set input mean to 0 over the dataset.

In [21]:
import numpy as np
gen.mean = np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

In [22]:
for layer in vgg16.layers:
    print(layer.input)
#     print(layer.input_shape) # channels_last
#     print(layer.output_shape) # channels_last
# 此处第一第二层干了什么事情？不是很明白呢

Tensor("input_2:0", shape=(?, 150, 150, 3), dtype=float32)
Tensor("input_2:0", shape=(?, 150, 150, 3), dtype=float32)
Tensor("block1_conv1_1/Relu:0", shape=(?, 150, 150, 64), dtype=float32)
Tensor("block1_conv2_1/Relu:0", shape=(?, 150, 150, 64), dtype=float32)
Tensor("block1_pool_1/MaxPool:0", shape=(?, 75, 75, 64), dtype=float32)
Tensor("block2_conv1_1/Relu:0", shape=(?, 75, 75, 128), dtype=float32)
Tensor("block2_conv2_1/Relu:0", shape=(?, 75, 75, 128), dtype=float32)
Tensor("block2_pool_1/MaxPool:0", shape=(?, 37, 37, 128), dtype=float32)
Tensor("block3_conv1_1/Relu:0", shape=(?, 37, 37, 256), dtype=float32)
Tensor("block3_conv2_1/Relu:0", shape=(?, 37, 37, 256), dtype=float32)
Tensor("block3_conv3_1/Relu:0", shape=(?, 37, 37, 256), dtype=float32)
Tensor("block3_pool_1/MaxPool:0", shape=(?, 18, 18, 256), dtype=float32)
Tensor("block4_conv1_1/Relu:0", shape=(?, 18, 18, 512), dtype=float32)
Tensor("block4_conv2_1/Relu:0", shape=(?, 18, 18, 512), dtype=float32)
Tensor("block4_conv3_1/

In [23]:
# layer.input_shape
# layer.output_shape
batch_size = 8

In [24]:
generator = gen.flow_from_directory(
        train_path,
        target_size=(image_witdth, image_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 2000 images belonging to 2 classes.


In [25]:
print(generator.class_indices)
print(generator.samples)
print(generator.batch_size)

{'cats': 0, 'dogs': 1}
2000
8


In [26]:
# 序
bottleneck_features_train = vgg16.predict_generator(
        generator, steps = generator.samples // generator.batch_size)

ResourceExhaustedError: OOM when allocating tensor with shape[8,150,150,64]
	 [[Node: block1_conv1_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_2_0/_209, block1_conv1_1/kernel/read)]]
	 [[Node: block5_pool_1/MaxPool/_211 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_59_block5_pool_1/MaxPool", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'block1_conv1_1/convolution', defined at:
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/ipyk